In [2]:
from almanac.data.data import Read_csv


In [3]:
from almanac.analysis.calculate_returns import calculate_perc_returns

In [9]:
from almanac.utils.utils import *

from almanac.analysis.calculate_stats import Stats

In [5]:
reader = Read_csv(filename='/home/ornob/Documents/sp500.csv', )

In [6]:
data = reader.pd_readcsv()
data

,adjusted,underlying
1982-09-14,405.40,123.70
1982-09-15,405.90,124.20
1982-09-16,405.25,123.55
1982-09-17,404.35,122.65
1982-09-20,405.15,123.45
...,...,...
2022-09-26,3667.25,3667.25
2022-09-27,3665.25,3665.25
2022-09-28,3729.75,3729.75
2022-09-29,3658.75,3658.75


In [7]:
adjusted_price = data.adjusted
current_price = data.underlying
multiplier = 5
fx_series = pd.Series(1, index=data.index)  
position_contracts_held = pd.Series(1, index=data.index)  

capital_required = multiplier * current_price 


In [8]:
perc_return = calculate_perc_returns(
    position_contracts_held=position_contracts_held,
    adjusted_price = adjusted_price,
    fx_series=fx_series,
    capital_required=capital_required,
    multiplier=multiplier
)



In [25]:
a = Stats(perc_return)

In [22]:
a.stats()

In [14]:
stats.sharpe

0.2567531397519812

In [23]:
a.stats(show=True)

TypeError: 'Series' object is not callable

In [26]:
a.show_stats()

{'ann_mean': 0.05090927903394909,
 'ann_std': 0.19688421110728013,
 'sharpe_ratio': <bound method Stats.sharpe_ratio of <almanac.analysis.calculate_stats.Stats object at 0x7f1d78909ed0>>,
 'skew': -3.6436202034435747,
 'avg_drawdown': 0.23304463812166343,
 'max_drawdown': 1.2141040208288036,
 'quant_ratio_lower': 2.2302136448277934,
 'quant_ratio_upper': 1.6017921571611629}